In [1]:
from sage.all import *
import sys
import hashlib
import random
import math


In [2]:
# load('./keygen_impl.sage')

In [2]:
# n is the ring degree
def keygen(k,n,l,r,m,t):
    #random.seed(seed)
    U = random_matrix(GF(2), n-k,l)
    B = random_matrix(GF(2), n-k,n)
    show(U)
    show(B)
   
    #constructing trapdoor A
    
    preparse("K1_ =  GF(2, 'a_0')")
    K1_ = GF(2, 'a_0')
    preparse("F1 =  GF(n, 'a')")
    F1 = GF(n, 'a')
    
    preparse("PR1.<X> = PolynomialRing(F1)")
    PR1 = PolynomialRing(F1, names=('X',))
    #PR = PolynomialRing(F,'X')
    X = PR1.gen()
    a_0 = K1_.gen()
    a = F1.gen() # define the generator of the field F
    L1 = [F1(a**i) for i in range(2048)]
   

    # Irreducible polynomial generation

    while True:
        irr_poly = PR1.random_element(t-1)
        irr_poly += X**t
        if irr_poly.is_irreducible():
            break
    g1 = irr_poly
    show(g1)

    h = PR1(1)
    for a_i in L1:
        h = h * (X - a_i);

    gamma = [];

    for a_i in L1:
        gamma.append((h * ((X - a_i).inverse_mod(g1))).mod(g1));
        
    H_check_poly = matrix(F1, t, n);
    for i in range(n):
        coeffs = list(gamma[i]);
        for j in range(t):
            # Check to make sure the coefficient exists
            # (It may not if the polynomial is not of
            # degree t)
            if j < len(coeffs):
                H_check_poly[j, i] = coeffs[j];
            else:
                H_check_poly[j, i] = F1(0);

      # Construct the binary parity-check matrix for the Goppa code.
        # Do so by converting each element of F_2**m to its binary
        # representation.
    H_Goppa_A = matrix(K1_, 11 * H_check_poly.nrows(), H_check_poly.ncols());
    for i in range(H_check_poly.nrows()):
        for j in range(H_check_poly.ncols()):
            be = bin(eval(H_check_poly[i, j]._int_repr()))[2:];
#             be = bin(eval(H[i,j]._int_repr()))[2:];
            be = '0'*(11 - len(be)) + be;
            be = list(be);
            H_Goppa_A[11 * i:11 * (i + 1), j] = vector(map(int, be));

        # Construct the generator matrix for our code by computing
        # a basis for the null-space of H_Goppa_A. The null-space is,
        # by definition, the codewords of our code.
        
        #G_Goppa = H_Goppa_A.right_kernel().basis_matrix();
     
    print("Irreducible polynomial g(x):", g1)
    S_A = random_matrix(GF(2), H_Goppa_A.nrows())
    while (S_A.determinant()==0):
        S_A = random_matrix(GF(2), H_Goppa_A.nrows())

    rng = range(n)

    P_A = matrix(GF(2),n);

    for i in range(n):
        p = floor(len(rng)*random.random());
        P_A[i,rng[p]]=1;
        rng=[*rng[:p], *rng[p+1:]];
    
    A = S_A*H_Goppa_A*P_A
   
   
    preparse("K_ =  GF(2, 'b')")
    K_ = GF(2, 'b')
    preparse("F = GF(m, 'b')")
    F = GF(m, 'b')
    preparse("PR.<Y> = PolynomialRing(F)")
    PR = PolynomialRing(F, names=('Y',))
    #PR = PolynomialRing(F,'X')
    Y = PR.gen()
    b = F.gen() # define the generator of the field F
    L = [F(b**i) for i in range(2048)]
    flag=0
    while flag ==0:
        # Irreducible polynomial generation

        
        while True:
            irr_poly = PR.random_element(t-1)
            irr_poly += Y**t
            if irr_poly.is_irreducible():
                break
        g = irr_poly
        show(g)


    #     # Key generation
    #     T = matrix(F,rango,rango)
    #     for i in range(rango):
    #         count = rango - i
    #         for j in range(rango):
    #             if i > j:
    #                 T[i,j]=g.list()[count]
    #                 count = count + 1
    #             if i < j:
    #                 T[i,j] = 0
    #             if i == j:
    #                 T[i,j] = 1

    #     D = diagonal_matrix([1/g(L[i]) for i in range(N)])
    #     H = T*V*D
    #     H_Goppa_K = matrix(K_, 11*H.nrows(),H.ncols())
    #     #print ("Matriz H_Goppa_K inicial: ")
    #     show(H_Goppa_K)


        h = PR(1)
        for a_i in L:
            h = h * (Y - a_i);

        gamma = [];

        for a_i in L:
            gamma.append((h * ((Y - a_i).inverse_mod(g))).mod(g));

        H_check_poly = matrix(F, t, m);
        for i in range(n):
            coeffs = list(gamma[i]);
            for j in range(t):
                # Check to make sure the coefficient exists
                # (It may not if the polynomial is not of
                # degree t)
                if j < len(coeffs):
                    H_check_poly[j, i] = coeffs[j];
                else:
                    H_check_poly[j, i] = F(0);

      # Construct the binary parity-check matrix for the Goppa code.
        # Do so by converting each element of F_2**m to its binary
        # representation.
        H_Goppa = matrix(K_, 11 * H_check_poly.nrows(), H_check_poly.ncols());
        for i in range(H_check_poly.nrows()):
            for j in range(H_check_poly.ncols()):
                be = bin(eval(H_check_poly[i, j]._int_repr()))[2:];
    #             be = bin(eval(H[i,j]._int_repr()))[2:];
                be = '0'*(11 - len(be)) + be;
                be = list(be);
                H_Goppa[11 * i:11 * (i + 1), j] = vector(map(int, be));

        # Construct the generator matrix for our code by computing
        # a basis for the null-space of H_Goppa. The null-space is,
        # by definition, the codewords of our code.
        G_Goppa = H_Goppa.right_kernel().basis_matrix();
        show(G_Goppa)
        show(H_Goppa)



        #print("Parity check matrix H:\n", H)
        V = matrix(GF(2), [H_Goppa.columns()[i] for i in range(H_Goppa.nrows())]).transpose()
        if V.det()==0:
            continue       
        V_ = matrix(K_, [G_Goppa.columns()[i] for i in range(G_Goppa.nrows())])
        if V_.det()==0:
            continue
        flag=1
    print("Irreducible polynomial g(x):", g)
    S = random_matrix(GF(2), r)
    while (S.determinant()==0):
        S = random_matrix(GF(2), r)

    rng = range(m)

    P = matrix(GF(2),m);

    for i in range(m):
        p = floor(len(rng)*random.random());
        P[i,rng[p]]=1;
        rng=[*rng[:p], *rng[p+1:]];
   
    G_pub = S*G_Goppa*P


    return (U,B,A,G_pub),(S_A,P_A,a_0,a,g1,S,P,b,g)


In [28]:
pk,gmsk=keygen(1696,2048,170,1696,2048,32)

352 x 170 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)
352 x 2048 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)
X^32 + (a^10 + a^9 + a^7 + a^6 + a^5 + a + 1)*X^31 + (a^10 + a^8 + a^7 + a^6 + 1)*X^30 + (a^8 + a^6 + a^4 + a^3 + a^2 + a)*X^29 + (a^10 + a^6 + a^5 + a^3 + 1)*X^28 + (a^10 + a^9 + a^8 + a^4 + a)*X^27 + (a^9 + a^8 + a^7 + a^5 + a^4 + a^3 + a^2 + a + 1)*X^26 + (a^10 + a^8 + a^7 + a^6 + a^4 + a)*X^25 + (a^10 + a^4 + a^2)*X^24 + (a^6 + a^5 + a^3 + a^2 + a)*X^23 + (a^9 + a^5 + a^4 + a^3 + a + 1)*X^22 + (a^10 + a^6 + a^4 + a^2 + a + 1)*X^21 + (a^8 + a^7 + a^6 + a^5 + a^4 + a^2 + a + 1)*X^20 + (a^10 + a^9 + a^8 + a^7 + a^6 + a^5 + a^4 + a^3 + a^2 + a + 1)*X^19 + (a^9 + a^8 + a^7 + a^6 + a + 1)*X^18 + (a^9 + a^6 + a^5 + a^4 + a^3 + a^2 + a + 1)*X^17 + (a^9 + a^8 + a^7 + a^6 + a^4 + a^3 + a^2 + 1)*X^16 + (a^10 + a^8 + a^5 + a^4 + a^2 + 1)*X^15 + (a^10 + a^9 + a^7 + a^6 + a^5)*X^14 + (a^9 + a^3 + a^2 + a +

KeyboardInterrupt: 

In [5]:
pk

(352 x 170 dense matrix over Finite Field of size 2,
 352 x 2048 dense matrix over Finite Field of size 2,
 352 x 2048 dense matrix over Finite Field of size 2,
 1696 x 2048 dense matrix over Finite Field of size 2)

In [6]:
gmsk

(352 x 352 dense matrix over Finite Field of size 2,
 2048 x 2048 dense matrix over Finite Field of size 2,
 1,
 a,
 X^32 + (a^10 + a^4 + a^2 + a + 1)*X^31 + (a^10 + a^8 + a^7 + a^6 + a^2)*X^30 + (a^10 + a^8 + a^6 + a^3 + a + 1)*X^29 + (a^10 + a^9 + a^8 + a^7 + a^6 + a^4 + a^3 + a + 1)*X^28 + (a^10 + a^9 + a^7 + a^4 + 1)*X^27 + (a^10 + a^9 + a^7 + a^6 + a^5 + a^2 + a + 1)*X^26 + (a^10 + a^8 + a^7 + a^5)*X^25 + (a^10 + a^9 + a^6 + a^4 + a^3 + a^2 + 1)*X^24 + (a^10 + a^9 + a^8 + a^7 + a^4 + a^3)*X^23 + (a^10 + a^9 + a^8 + a^6 + 1)*X^22 + (a^9 + a^8 + a^6)*X^21 + (a^8 + a^7 + a^5 + a^4 + a^3)*X^20 + (a^10 + a^9 + a^5 + a^3 + a^2 + a)*X^19 + (a^9 + a^6 + a^4 + a)*X^18 + (a^10 + a^7 + a^6 + a^4)*X^17 + (a^7 + a^4 + a + 1)*X^16 + (a^8 + a)*X^15 + (a^10 + a^9 + a^5 + a^4 + a^2 + a + 1)*X^14 + (a^9 + a^6 + a^4 + a^3)*X^13 + (a^10 + a^9 + a^8 + a^7 + a^5 + a^3 + a^2 + 1)*X^12 + (a^10 + a^8 + a^7 + a)*X^11 + (a^10 + a^7 + a^5 + a^4 + a^2)*X^10 + (a^10 + a^5 + a^4 + a^3 + a)*X^9 + (a^9 + a^8 + a^

In [7]:
# pk[0]

In [8]:
# show(pk[0].str())

In [9]:
from sys import getsizeof as getsize

In [10]:
print(sys.getsizeof(pk[0]))

144


In [11]:
print(getsize(pk[0].str()))
print(getsize(pk[1].str()))
print(getsize(pk[2].str()))
print(getsize(pk[3].str()))

120432
1442544
1442544
6950256


In [12]:
# print(pk[0].nbits())

In [3]:
key_id = '0'
data_path = './'
key_suffix = '80' + '_' + key_id


filename = data_path + 'sk' + key_suffix
print('secret key saved in file ' + filename + '.sobj')
save(gmsk, filename)

filename = data_path + 'pk' 
print ('public key saved in file ' + filename + '.sobj')
save(pk, filename)

secret key saved in file ./sk80_0.sobj


NameError: name 'gmsk' is not defined

In [2]:
key_id = '0'
data_path = './'
key_suffix = '80' + '_' + key_id
gmsk = load(data_path + 'sk' + key_suffix)
pk = load(data_path + 'pk')

In [15]:
# a=gmsk[2]

In [3]:
def gen_binom_vec(n, tau, K_):
    v= [1 if random.random()<tau else 0 for _ in range(n)]
    v = vector(K_,v)
    return v

In [4]:
def algoritmo_euclides_extendido(self, other):
    delta = self.degree()

    if other.is_zero():
        ring = self.parent()
        return self, R.one(), R.zero()

    ring = self.parent()
    a = self
    b = other

    s = ring.one()
    t = ring.zero()

    resto0 = a
    resto1 = b

    while true:
        cociente,resto_auxiliar = resto0.quo_rem(resto1)
        resto0 = resto1
        resto1 = resto_auxiliar

        s = t
        t = s - t*cociente

        if resto1.degree() <= floor((delta-1)/2) and resto0.degree() <= floor((delta)/2):
             break

    V = (resto0-a*s)//b
    coeficiente_lider = resto0.leading_coefficient()

    return resto0/coeficiente_lider, s/coeficiente_lider, V/coeficiente_lider


def descomponer_polinomio(p):

    Phi1 = p.parent()

    p0 = Phi1([sqrt(c) for c in p.list()[0::2]])
    p1 = Phi1([sqrt(c) for c in p.list()[1::2]])
    return (p0,p1)

def inversa_g(p,g):
    (d,u,v) = xgcd(p,g)
    return u.mod(g)

def _norm(a, b,X):
    # This is the way in which Bernstein indicates
    # the norm of a member of the lattice is
    # to be defined.
    return 2 ** ((a ** 2 + X * b ** 2).degree());

def _lattice_basis_reduce(s,g,X):
    t = g.degree();

    a = [];
    a.append(0);
    b = [];
    b.append(0);
    (q, r) = g.quo_rem(s);
    (a[0], b[0]) = simplify((g - q * s, 0 - q))

    # If the norm is already small enough, we
    # are done. Otherwise, intialize the base
    # case of the recursive process.
    if _norm(a[0], b[0],X) > 2 ** t:
        a.append(0);
        b.append(0);
        (q, r) = s.quo_rem(a[0]);
        (a[1], b[1]) = (r, 1 - q * b[0]);
    else:
        return (a[0], b[0]);

    # Continue subtracting integer multiples of
    # the shorter vector from the longer until
    # the produced vector has a small enough norm.
    i = 1;
    while _norm(a[i], b[i],X) > 2 ** t:
        a.append(0);
        b.append(0);
        (q, r) = a[i - 1].quo_rem(a[i]);
        (a[i + 1], b[i + 1]) = (r, b[i - 1] - q * b[i]);
        i += 1;

    return (a[i], b[i]);
    

def decodePatterson(y,g,X,SyndromeCalculator,L):

    synd = SyndromeCalculator * y.transpose();
    syndrome_poly = 0;
    for i in range(synd.nrows()):
        syndrome_poly += synd[i, 0] * X ** i

    vector_g = descomponer_polinomio(g)
    w = ((vector_g[0])*inversa_g(vector_g[1],g)).mod(g)
    vector_t = descomponer_polinomio(inversa_g(syndrome_poly,g) + X)

    R = (vector_t[0]+(w)*(vector_t[1])).mod(g)

 # Perform lattice basis reduction.
    (alpha, beta) = _lattice_basis_reduce(R,g,X);

    # Construct the error-locator polynomial.
    sigma = (alpha * alpha) + (beta * beta) * X;

    # For every root of the error polynomial,
    # correct the error induced at the corresponding index.
    for i in range(len(L)):
        if sigma(L[i]) == 0:
            y[0, i] += 1;
    return y;


# def decodePatterson(y,g,X,H_check_poly,L):

# #     alpha = vector(H*y)

# #     polinomioS = PR(0)
# #     for i in range(len(alpha)):
# #         polinomioS = polinomioS + alpha[i]*(X**(len(alpha)-i-1))

#     alpha = vector(H_check_poly*y.transpose())
#     polinomioS = PR(0)
#     for i in range(len(alpha)):
# #         polinomioS = polinomioS + alpha[i]*(X**(i))
#         polinomioS = polinomioS + alpha[i]*(X**(len(alpha)-i-1))



#     vector_g = descomponer_polinomio(g)
#     w = ((vector_g[0])*inversa_g(vector_g[1],g)).mod(g)
#     vector_t = descomponer_polinomio(inversa_g(polinomioS,g) + X)

#     R = (vector_t[0]+(w)*(vector_t[1])).mod(g)

#     (a11,b11,c11) = algoritmo_euclides_extendido(g,R)


#     sigma = a11**2+X*(c11**2)

# #     print(len(L))
#     for i in range(len(L)):
#         if (sigma(L[i])==0):
# #             print ("Error encontrado en la posición: " + str(i))
#              y[0, i] =  y[0, i] + 1
#     return y

In [5]:
def wt(x):
    x = vector(x)
    count = 0
    for i in range(len(x)):
        if x[i] != 0:
            count = count + 1
    return count

In [30]:
Join(pk,gmsk,2048,170,0.16,1)

'[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\

'[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\

'[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\

'[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\

'[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\

...

'[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\

'[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\

'[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\

'[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[0]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[0]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[0]\n[0]\n[0]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[0]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[1]\n[1]\n[1]\n[1]\n[0]\n[0]\n[1]\n[0]\n[1]\n[0]\

KeyboardInterrupt: 

In [6]:
n = 2048
l = 170
sigma_1 = 0.16
U = pk[0]
B = pk[1]
A = pk[2]

S_A = gmsk[0]
P_A = gmsk[1]
a_0 = gmsk[2]
a = gmsk[3]
g1 = gmsk[4]
#     show(U)
#     show(B)
#     show(A)
#     show(g1)

F = a.parent()
PR = g1.parent()
X = PR.gen()
K_ = a_0.parent()
L = [F(a**i) for i in range(2048)]
SyndromeCalculator = matrix(PR, 1, len(L));
for i in range(len(L)):
    SyndromeCalculator[0, i] = (X - L[i]).inverse_mod(g1);

h = PR(1)
for a_i in L:
    h = h * (X - a_i);
gamma = [];

for a_i in L:
    gamma.append((h * ((X - a_i).inverse_mod(g1))).mod(g1));

t = 32    
H_check_poly = matrix(F, t, n);
for i in range(n):
    coeffs = list(gamma[i]);
    for j in range(t):
        # Check to make sure the coefficient exists
        # (It may not if the polynomial is not of
        # degree t)
        if j < len(coeffs):
            H_check_poly[j, i] = coeffs[j];
        else:
            H_check_poly[j, i] = F(0);

# Construct the binary parity-check matrix for the Goppa code.
# Do so by converting each element of F_2**m to its binary
# representation.
H_Goppa = matrix(K_, 11 * H_check_poly.nrows(), H_check_poly.ncols());
for i in range(H_check_poly.nrows()):
    for j in range(H_check_poly.ncols()):
        be = bin(eval(H_check_poly[i, j]._int_repr()))[2:];
#             be = bin(eval(H[i,j]._int_repr()))[2:];
        be = '0'*(11 - len(be)) + be;
        be = list(be);
        H_Goppa[11 * i:11 * (i + 1), j] = vector(map(int, be));

In [7]:
G_Goppa = H_Goppa.right_kernel().basis_matrix();


In [8]:
def gen_random_fix_vec(n, w):
    vec = [1]*w+(n-w)*[0]
    random.shuffle(vec)
    return vec

e=gen_random_fix_vec(n,16)
e=vector(GF(Integer(2)),e)

In [59]:
u = vector(K_,[randint(0,1) for _ in range(G_Goppa.nrows())])
c = u*G_Goppa
y = c + e
yd = decodePatterson(matrix(K_,y),g1,X,SyndromeCalculator,L)
e_prime = y + vector(K_,yd.list())
show(e_prime)

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [44]:
show(vector(H_Goppa*yd.transpose()))

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [53]:
vec = gen_binom_vec(n, sigma_1)
s_prime = U[:,0] - B*matrix(GF(2),vec).transpose()
y_p = H_Goppa.solve_right(s_prime)
y_d = decodePatterson(y_p.transpose(),g1,X,SyndromeCalculator,L)
show(vector(H_Goppa*yd.transpose()))

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [133]:
show(vector(y_p + yd.transpose()))
print(wt(vector(y_p + yd.transpose())))
show(vector(y_p))

TypeError: unsupported operand parent(s) for +: 'Vector space of dimension 2048 over Finite Field of size 2' and 'Full MatrixSpace of 2048 by 1 dense matrices over Finite Field of size 2'

In [59]:
P_prime = matrix(GF(2),2048);
rng = range(2048)
for i in range(2048):
    p = floor(len(rng)*random.random());
    P_prime[i,rng[p]]=1;
    rng=[*rng[:p], *rng[p+1:]];

H_prime = H_Goppa * P_prime

In [120]:
# U = random_matrix(K_, 352,170)
# B = random_matrix(K_, 352,2048)

In [ ]:
flag = 0
while flag ==0:
    vec = gen_binom_vec(n, 0.5)
#     s_prime = U[:,2] + B*matrix(GF(2),vec).transpose()
    u = vector(K_,[randint(0,1) for _ in range(G_Goppa.nrows())])
    c = u * G_Goppa
    s_0 = H_Goppa * matrix(K_,e).transpose()
    
    s_prime = U[:,2] + B*matrix(K_,vec).transpose()
#     for i in range(len(vector(s_prime))):
#         if vector(s_prime)[i] != 0:
#             s_0[i] = 1
#         else :
#             s_0[i] = 0
            
    y_p = H_Goppa.solve_right(s_prime)
#     y_p = P_prime * y_p

#     show((s_prime - H_Goppa  * y_p).str())
    y_p = c + vector(K_,y_p.list())
#     y_p = vector(K_,y_p.list())
    y_d = decodePatterson(matrix(K_,y_p),g1,X,SyndromeCalculator,L)
#     show((H_Goppa  * matrix(K_,c).transpose()).str())
#     show((s_prime - H_Goppa  * matrix(K_,y_p).transpose()).str())
#     show(vector(H_Goppa*yd.transpose()))
#     show(vector(y_p + yd.transpose()))
    print(wt(vector(y_p + vector(K_,y_d.list()))))
#     y_p2 = c + vector(K_,y_p.list()) + e
#     y_d2 = decodePatterson(matrix(K_,y_p2),g1,X,SyndromeCalculator,L)
#     print(wt(vector(y_p2 + vector(K_,y_d2.list()))))
#     show(vector(y_p))
    if(wt(vector(y_p + vector(K_,y_d.list()))) <= 64 and wt(vector(y_p + vector(K_,y_d.list()))) >= 10):
        flag = 1
        show(vector(y_p + vector(K_,y_d.list())))

0
1
1
0
1
2
1
3
1
1
1
0
0
2
1
0
1
1
1
2
0
2
0
1
1
0
0
0
0
2
0
2
3
1
0
0
1
0
1
0
1
2
1
3
0
1
1
1
1
2
0
1
1
0
0
0
0
1
1
4
0
0
1
1
1
0
2
0
1
1
0
1
0
1
3
1
0
1
2
0
1
3
1
0
0
2
2
0
0
2
1
3
1
1
0
0
0
0
1
0
1
1
0
2
2
0
2
1
2
1
2
3
1
0
1
1
1
1
0
1
0
1
0
0
1
0
0
1
2
3
0
0
2
1
2
0
2
0
0
0
2
1
2
0
0
1
0
0
1
1
0
2
0
1
3
1
1
2
2
1
1
2
1
0
2
2
0
0
1
0
3
1
1
1
2
1
1
0
0
3
0
2
2
2
2
0
0
2
0
1
0
1
2
0
0
1
1
0
0
1
1
1
0
2
0
0
3
1
1
0
2
1
0
0
1
2
1
3
1
2
0
0
5
0
1
0
1
1
1
0
2
2
0
0
0
0
1
1
3
1
4
0
1
0
4
1
0
0
2
0
2
3
0
1
1
0
0
0
0
0
1
1
1
3
1
1
2
1
0
2
2
3
0
1
1
1
3
2
1
0
0
2
0
0
2
0
1
0
6
3
0
2
0
1
1
2
0
2
1
1
0
0
0
2
1
1


In [9]:
# random permutation matrix
columns = list(range(n))
P3 = matrix(K_, n)

for i in range(n):
    tt = randint(0, len(columns)-1)
    j = columns[tt]
    P3[i,j] = 1
    columns.remove(j)

In [12]:
k=1696
l=170
U = matrix(K_, n-k,l)
B = matrix(K_, n-k,n)
# A = NIE.get_public_key()
B = A * P3
for j in range(l):
    e_j = random_error(2048, 10, K_)
    c_j = e_j * A.transpose()
    U[:,j] = c_j

In [11]:
def random_error(n, num_errors, field):
    """
    Return a random vector of length ``n`` with maximum weight 
    ``num_errors`` belonging to the finite field ``field``
    
    INPUT:
        - ``n`` -- length
        - ``num_errors`` -- maximum weight
        - ``field`` -- finite field
    """
    v = [0] * n
    i = 0
    
    while i < num_errors:
        m = randint(0, len(v)-1)
        v[m] = choice(field.list())
        i += 1

    e = vector(field, v)
    
    return e

In [48]:
# def PattersonDecode_to_code(y,g,X,SyndromeCalculator,L):
#     yy = matrix(K_,y)
#     synd = SyndromeCalculator * yy.transpose();
#     syndrome_poly = 0;
#     for i in range(synd.nrows()):
#         syndrome_poly += synd[i, 0] * X ** i

#     vector_g = descomponer_polinomio(g)
#     w = ((vector_g[0])*inversa_g(vector_g[1],g)).mod(g)
#     vector_t = descomponer_polinomio(inversa_g(syndrome_poly,g) + X)

#     R = (vector_t[0]+(w)*(vector_t[1])).mod(g)

#  # Perform lattice basis reduction.
#     (alpha, beta) = _lattice_basis_reduce(R,g,X);

#     # Construct the error-locator polynomial.
#     sigma = (alpha * alpha) + (beta * beta) * X;

#     # For every root of the error polynomial,
#     # correct the error induced at the corresponding index.
#     for i in range(len(L)):
#         if sigma(L[i]) == 0:
#             y[i] += 1;
#     return y;


In [13]:
def DecodeNiederreiter(c,_S,_P,_H,g,X,SyndromeCalculator,L):
    
#         show(c * (self._S).inverse().transpose())
        word = (_H).solve_right(c * (_S).inverse().transpose())
#         show(word)
#         word = c * self._P.transpose()
        word_mat = matrix(K_,word)
#         yd = decodePatterson(word_mat,g1,X,SyndromeCalculator,L)
        word_prime = decodePatterson(word_mat,g1,X,SyndromeCalculator,L)
#         show(word_prime)
        word_prime = vector(K_,word_prime.list())
        message = word - word_prime
        message = message * _P
#         show(message)
        return message

In [14]:
def get_weight(c):
    """
    Return the weight of ``c``
    
    INPUT:
        - ``c`` -- a vector
    """
    w = 0
    
    for elem in c:
        if elem != 0:
            w += 1
    
    return w

In [15]:
X_1 = matrix(K_, n,l)
X_2 = matrix(K_, n,l)

In [16]:
for j in range(l):
    flag = 0
    while flag == 0:
        u_j = vector(K_,U[:,j].list())
        x_1j = random_error(2048, 20, K_)
        while get_weight(x_1j) < 20:
            i = randint(0, len(e) - 1)
            x_1j[i] = 1
        v_j = u_j + x_1j * B.transpose()
        v_j = vector(K_,v_j)
        x_2j = DecodeNiederreiter(v_j,S_A,P_A,H_Goppa,g1,X,SyndromeCalculator,L)
#         show(x_2j)
        if (v_j ==  x_2j * A.transpose()):
            flag = 1
            print(get_weight(x_2j))
#             show(message_random)
#             show(message3)
            X_1[:,j] = x_1j
            X_2[:,j] = x_2j
    
            

24
26
25
26
25
25
26
28
25
23
26
24
24
24
26
28
24
24
25
26
22
24
25
24
23
27
25
25
25
26
23
23
24
26
23
25
27
27
24
20
26
23
27
24
25
25
28
26
26
25
24
25
23
25
26
23
23
24
23
24
27
24
28
24
24
23
23
27
28
27
27
24
26
26
22
23
24
25
24
24
25
24
26
26
26
25
25
25
25
25
25
24
25
26
25
28
24
27
26
27
25
25
24
26
26
25
23
27
28
26
23
24
25
24
26
25
24
22
24
24
21
27
25
25
28
27
24
24
26
24
23
26
24
22
26
24
26
26
25
26
26
23
23
24
23
25
25
26
25
25
26
24
25
29
25
24
24
28
28
22
22
26
24
23
26
26
27
23
27
26


In [64]:
show(X_1)

2048 x 170 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)


In [65]:
show(X_2)

2048 x 170 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)


In [66]:
show(X_1.str())

'[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [18]:
B * X_1 + A * X_2 == U

True

In [23]:
gpk = B * X_1
gsk = (X_1,X_2)

In [24]:
key_id = '0'
data_path = './'
key_suffix = '80' + '_' + key_id


filename = data_path + 'gsk' + key_suffix
print('secret key saved in file ' + filename + '.sobj')
save(gsk, filename)

filename = data_path + 'gpk' + key_suffix
print ('public key saved in file ' + filename + '.sobj')
save(gpk, filename)

secret key saved in file ./gsk80_0.sobj
public key saved in file ./gpk80_0.sobj


In [22]:
key_id = '0'
data_path = './'
key_suffix = '80' + '_' + key_id


gpk = load(data_path + 'gpk'+ key_suffix)
gsk = load(data_path + 'gsk'+ key_suffix)

In [19]:
pk = (U,B,A,pk[3])

In [20]:
key_id = '0'
data_path = './'
key_suffix = '80' + '_' + key_id


filename = data_path + 'pk' 
print ('public key saved in file ' + filename + '.sobj')
save(pk, filename)

public key saved in file ./pk.sobj


In [26]:
#成员加入过程
def Join(U,B,A,_S,_P,_H,g,X,SyndromeCalculator,L,n,l,sigma_1,U_i):
#     #random.seed(seed)
#     U = pk[0]
#     B = pk[1]
#     A = pk[2]
   
#     S_A = gmsk[0]
#     P_A = gmsk[1]
#     a_0 = gmsk[2]
#     a = gmsk[3]
#     g1 = gmsk[4]
# #     show(U)
# #     show(B)
# #     show(A)
# #     show(g1)
    
#     F = a.parent()
#     PR = g1.parent()
#     X = PR.gen()
#     K_ = a_0.parent()
#     L = [F(a**i) for i in range(2048)]
#     SyndromeCalculator = matrix(PR, 1, len(L));
#     for i in range(len(L)):
#         SyndromeCalculator[0, i] = (X - L[i]).inverse_mod(g1);
        
#     h = PR(1)
#     for a_i in L:
#         h = h * (X - a_i);
#     gamma = [];

#     for a_i in L:
#         gamma.append((h * ((X - a_i).inverse_mod(g1))).mod(g1));
        
#     t = 32    
#     H_check_poly = matrix(F, t, n);
#     for i in range(n):
#         coeffs = list(gamma[i]);
#         for j in range(t):
#             # Check to make sure the coefficient exists
#             # (It may not if the polynomial is not of
#             # degree t)
#             if j < len(coeffs):
#                 H_check_poly[j, i] = coeffs[j];
#             else:
#                 H_check_poly[j, i] = F(0);

#   # Construct the binary parity-check matrix for the Goppa code.
#     # Do so by converting each element of F_2**m to its binary
#     # representation.
#     H_Goppa = matrix(K_, 11 * H_check_poly.nrows(), H_check_poly.ncols());
#     for i in range(H_check_poly.nrows()):
#         for j in range(H_check_poly.ncols()):
#             be = bin(eval(H_check_poly[i, j]._int_repr()))[2:];
# #             be = bin(eval(H[i,j]._int_repr()))[2:];
#             be = '0'*(11 - len(be)) + be;
#             be = list(be);
#             H_Goppa[11 * i:11 * (i + 1), j] = vector(map(int, be));

    
    
    X_1 = matrix(K_, n, l)
    X_2 = matrix(K_, n, l)
    for j in range(l):
        flag = 0
        while flag == 0:
            u_j = vector(K_,U[:,j].list())
            x_1j = gen_binom_vec(n, sigma_1,K_)
    #         while get_weight(x_1j) < 20:
    #             i = randint(0, len(e) - 1)
    #             x_1j[i] = 1
            v_j = u_j + x_1j * B.transpose()
            v_j = vector(K_,v_j)
            x_2j = DecodeNiederreiter(v_j,_S,_P,_H,g,X,SyndromeCalculator,L)
    #         show(x_2j)
            if (v_j ==  x_2j * A.transpose()):
                flag = 1
#                 print(get_weight(x_2j))
    #             show(message_random)
    #             show(message3)
                X_1[:,j] = x_1j
                X_2[:,j] = x_2j
#                 show(X_2[:,j].transpose().str())
#    show(X_1.str())
        
    
#     V = U - B*X_1
#     X_2 = matrix(GF(2), n, l)
#     for j in range(l):
#         s_prime = V[:,j]
#         y_p = A.solve_right(S_A.inverse()*s_prime)
# #         show(y_p.str())
#         y_d = decodePatterson(y_p.transpose(),g1,X,SyndromeCalculator,L)
#         X_2[:,j] = P_A.transpose() * (y_p - y_d.transpose())
#         show(X_2[:,j].transpose().str())
    
#      show(X_2.str())
    
    return B*X_1,(X_1,X_2)
        
    
    
    

    
   

In [78]:
gpk,gsk = Join(U,B,A,S_A,P_A,H_Goppa,g1,X,SyndromeCalculator,L,2048,170,0.01,1)

24
23
28
24
32
28
26
30
24
21
31
24
22
24
22
21
28
16
31
25
16
26
28
21
29
26
31
25
17
32
29
21
25
24
28
14
25
24
25
24
31
23
32
28
32
26
25
28
27
26
27
30
22
21
26
28
31
22
20
24
29
28
20
25
32
19
29
27
25
20
28
28
25
23
31
26
25
26
18
22
23
32
28
31
19
26
21
28
26
19
26
28
23
27
27
24
20
26
28
20
29
21
23
28
30
30
27
28
31
26
24
30
21
20
31
25
26
31
16
27
17
30
26
30
22
23
27
32
25
26
27
31
21
25
21
20
21
22
26
24
27
17
31
18
30
24
22
22
20
24
30
25
32
18
18
26
21
16
22
10
30
30
27
25
32
25
25
23
21
23


In [25]:
gpklst = []
gsklst = []
N = 2 ** 6

In [29]:
for iter in range(N):
    gpk,gsk = Join(U,B,A,S_A,P_A,H_Goppa,g1,X,SyndromeCalculator,L,2048,170,0.01,iter)
    gpklst.append(gpk)
    gsklst.append(gsk)

In [30]:
key_id = '0'
data_path = './'
key_suffix = '80' + '_' + key_id


filename = data_path + 'gsklst' + key_suffix
print('secret key saved in file ' + filename + '.sobj')
save(gsklst, filename)

filename = data_path + 'gpklst' + key_suffix
print ('public key saved in file ' + filename + '.sobj')
save(gpklst, filename)

secret key saved in file ./gsklst80_0.sobj
public key saved in file ./gpklst80_0.sobj


In [31]:
len(gpklst)

64

In [32]:
user_i = randint(0,N-1)

1